In [29]:
import numpy as np
import json
import sys

with open('data.json') as json_file:
    data = json.load(json_file)

new_data = []
for p in data['data']:
    if p['t_type'] == 1:
        new_data.append(p)

        
patients = {}
city_raw = []
city = []

def checkIfAny(mainStr, listOfStr):
    for subStr in listOfStr:
        if subStr in mainStr:
            return (True, subStr)
    return (False, "")

            
def data_analysis():
    for p in new_data:
        start = p['t_pos_start'].replace(" ", "")
        end = p['t_pos_end'].replace(" ", "")
        if start not in city_raw:
            city_raw.append(start)
        if end not in city_raw:
            city_raw.append(end)

    city_raw.sort(key=len)
    city_raw.pop(0)

    for p in city_raw:
        if p not in city:
            result = checkIfAny(p, city)
            if result[0] == False:
                city.append(p)

    # print(city)

    for p in new_data:
        start = p['t_pos_start'].replace(" ", "")
        end = p['t_pos_end'].replace(" ", "")
        if start not in city:
            result = checkIfAny(start, city)
            if result[0]:
                start = result[1]

        if end not in city:
            result = checkIfAny(end, city)
            if result[0]:
                end = result[1]

        if start in patients:
            if end in patients[start]:
                patients[start][end] = patients[start][end] + 1
            else:
                patients[start][end] = 1
        else:
            patients[start] = {}
            patients[start][end] = 1

    # print(patients)

data_analysis()
    
point = [['温哥华', '北京', '广州'],['温哥华', '首尔', '广州'], ['温哥华', '广州']]
time = [[12, 5, 3],[13, 3, 4],[15]]
transfer_factor = 0.5

# print('加拿大温哥华: ' + str(patients['加拿大温哥华']))
# print('北京: ' + str(patients['北京']))
# print('首尔: ' + str(patients['首尔']))
# print('无锡: ' + str(patients['无锡']))
      

def check_if_contain(city, city_list):
    for c in city_list:
        if city in c:
            return (True, c)
    return (False, "")

def risk_during_flight(start, end):
    start = check_if_contain(start, city)
    end = check_if_contain(end, city)
    if (start[0] == True and end[0] == True and start[1] in patients and end[1] in patients[start[1]]):
        return patients[start[1]][end[1]]
    else:
        return 0
    
def risk_during_transfer(transfer):
    start = check_if_contain(transfer, city)
    end = check_if_contain(transfer, city)
    res = 0
    if start[0] == True:
        if start[1] in patients:
            res += int(sum(patients[start[1]][d] for d in patients[start[1]]) * transfer_factor)
#         print("start " + str(res))
    if end[0] == True:
        for s in patients:
            if end[1] in patients[s]:
#                 print(patients[s])
                res += int(patients[s][end[1]] * transfer_factor)
#         print("end " + str(res))
    return res

# print(risk_during_flight('加拿大温哥华', '北京'))
# print(risk_during_transfer('巴彦淖尔市临河区'))
# print(risk_during_flight('北京', '广州'))


def calculate_risk(point, time, transfer_factor): 
    risk = []
    whole_risk = []
    for i in range(len(point)):
        risk.append([])
        whole_risk.append(0)
        for s in range(len(point[i])-1):
            start = point[i][s]
            end = point[i][s+1]
            risk[i].append(risk_during_flight(start, end))
            if s != 0:
                risk[i].append(risk_during_transfer(start))
    #     print(risk[i])
        
        for s in range(len(time[i])):
            whole_risk[i] += risk[i][s] * time[i][s]
    #     print(whole_risk[i])

    for i in range(len(point)):
        for c in range(len(point[i])):
            if c != 0:
                print("-", end ="")
            print(point[i][c], end ="")
        print(": " + str(whole_risk[i]))
        
    length = sys.maxsize
    best = 0
    idx = -1
    while True:
        try:
            idx = whole_risk.index(min(whole_risk), idx+1)
            current_length = len(point[idx])
            if current_length < length:
                best = idx
                length = current_length
        except ValueError:
            break

#     best = whole_risk.index(min(whole_risk))
    
    print("best: ", end = "")
    for c in range(len(point[best])):
        if c != 0:
            print("-", end ="")
        print(point[best][c], end ="")

# calculate_risk(point, time, transfer_factor)

# print("best: ", end = "")
# for c in range(len(point[best])):
#     if c != 0:
#         print("-", end ="")
#     print(point[best][c], end ="")



In [30]:
import requests



# DCity = "YTO"
# ACity = "HKG"

def get_flights(D, A, Date):

    DCity = name_to_code(D)
    ACity = name_to_code(A)

    url = "https://us.trip.com/m/flights/api/oversea/secondListData/"

    headers = {
        "Content-Type": "application/json"
    }

    payload = {
        "Mode": 1,
        "SearchInfo": {
            "TripType": "OW",
            "CabinClass": "YSGroup",
            "TravelerNum": {
                "Adult": 1,
                "Child": 0,
                "Infant": 0
            },
            "SearchSegmentList": [
                {
                    "ACityCode": "",
                    "DCityCode": "",
                    "DDate": ""
                }
            ]
        },
        "FilterInfo": {
            "DPortCode": "",
            "APortCode": "",
            "AirlineCode": ""
        },
        "SegmentNo": 1,
        "requestHead": {
            "transactionID": "20200616210123259"
        }
    }

    payload["SearchInfo"]["SearchSegmentList"][0]["DCityCode"] = DCity
    payload["SearchInfo"]["SearchSegmentList"][0]["ACityCode"] = ACity
    payload["SearchInfo"]["SearchSegmentList"][0]["DDate"] = Date

    response = requests.request("POST", url, headers=headers, json=payload)

    flight_data = response.json()
    
    return flight_data


In [31]:
import csv
with open('iata.csv', newline='') as f:
    reader = csv.reader(f)
    next(reader)
    zipped = zip(*reader)
    iata = list(zipped)

def name_to_code(city):
    try:
        index = iata[1].index(city)
    except ValueError:
        return False
    return iata[5][index]

def code_to_name(code):
    try:
        index = iata[5].index(code)
    except ValueError:
        return False
    return iata[1][index]

In [32]:
from datetime import datetime
import math

def get_transfer(data):
    routes = []
    time = []
    for route_idx in range(len(data["responseContent"]["productInfoList"])):
        route = data["responseContent"]["productInfoList"][route_idx]
        route_new = []
        time_new = []
        for flight_idx in range(len(route["flightInfoList"])):
            flight = route["flightInfoList"][flight_idx]
            if flight_idx == 0:
                route_new.append(code_to_name(flight["dCityInfo"]["code"]))
            route_new.append(code_to_name(flight["aCityInfo"]["code"]))
            time_new.append(flight["durationInfo"]["hour"])
            if flight_idx != len(route["flightInfoList"]) - 1:
                last = datetime.strptime(route["flightInfoList"][flight_idx+1]["dDateTime"], '%Y-%m-%d %H:%M:%S')
                current = datetime.strptime(flight["aDateTime"], '%Y-%m-%d %H:%M:%S')
                difference = last - current
                transfer_time = difference.days * 24 + math.floor(difference.seconds / 3600)
                time_new.append(transfer_time)
        
        if route_new not in routes:
            routes.append(route_new)
            time.append(time_new)
        
    return routes, time



In [33]:
D = "多伦多"
A = "香港"
date = "2020-10-05"

flight_data = get_flights(D, A, date)

with open('flight.json', 'w') as json_file:
    json.dump(flight_data, json_file, indent=2)

routes, times = get_transfer(flight_data)

calculate_risk(routes, times, 0.5)

多伦多-达拉斯-香港: 0
多伦多-纽约-香港: 192
多伦多-奥克兰-香港: 0
多伦多-温哥华-香港: 13
多伦多-香港: 0
多伦多-波士顿-香港: 0
多伦多-杜拜-香港: 0
多伦多-漢城-香港: 0
多伦多-东京-香港: 16
多伦多-上海-香港: 60
多伦多-法兰克福-香港: 44
多伦多-芝加哥-东京-香港: 20
best: 多伦多-香港